# **Importing Libraries**

In [1]:
import gc
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)


In [2]:
#   Display Config
# ===============================
display_settings = {
    'display.max_columns': 200,
    'display.max_rows': 200,
    'display.expand_frame_repr': True,
    'display.precision': 4,
    'display.show_dimensions': True
}
for key, value in display_settings.items():
    pd.set_option(key, value)

print("Pandas settings applied.")

Pandas settings applied.


# [Drive Google Drive Link](https://drive.google.com/drive/folders/1yEbRemo_5eIuVM-Ps_ZQHOTPpi6yGUJQ?usp=drive_link)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Define main project path
# project_path = "/content/drive/MyDrive/DEPI_Project"

# **Load datasets**

In [4]:
# Paths to the raw data files
PRICES_PATH = "/kaggle/input/preprocessing/sell_prices.csv"
CALENDAR_PATH = "/kaggle/input/preprocessing/calendar.csv"
SALES_PATH = "/kaggle/input/preprocessing/sales_train_validation.csv"

# prices

In [5]:
dtypes_prices = {
    'store_id': 'str',
    'item_id': 'str',
    'wm_yr_wk': 'int16',
    'sell_price': 'float32'
}
cat_cols_prices = ['store_id', 'item_id']

df_prices = pd.read_csv(PRICES_PATH, dtype=dtypes_prices)
df_prices[cat_cols_prices] = df_prices[cat_cols_prices].astype('category')

print("Prices:", df_prices.shape)

Prices: (6841121, 4)


# calendar

In [6]:
#   Load Calendar
dtypes_calendar = {
    'date': 'str',
    'wm_yr_wk': 'int16',
    'weekday': 'str',
    'wday': 'int8',
    'month': 'int8',
    'year': 'int16',
    'd': 'str',
    'event_name_1': 'str',
    'event_type_1': 'str',
    'event_name_2': 'str',
    'event_type_2': 'str',
    'snap_CA': 'bool',
    'snap_TX': 'bool',
    'snap_WI': 'bool',
}
cat_cols_calendar = ['weekday', 'event_type_1', 'event_type_2']

df_calendar = pd.read_csv(CALENDAR_PATH, dtype=dtypes_calendar)
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_calendar[cat_cols_calendar] = df_calendar[cat_cols_calendar].astype('category')

print("Calendar:", df_calendar.shape)

Calendar: (1969, 14)


# Sales

In [7]:
#   Load Sales
horizon = 28
last_day = 1913

dtypes_sales = {
    'id': 'str',
    'item_id': 'str',
    'dept_id': 'str',
    'cat_id': 'str',
    'store_id': 'str',
    'state_id': 'str',
}
dtypes_sales_ts = {f'd_{day}': 'int16' for day in range(1, last_day)}
cat_cols_sales = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

df_sales = pd.read_csv(SALES_PATH, dtype=dict(dtypes_sales, **dtypes_sales_ts))
df_sales[cat_cols_sales] = df_sales[cat_cols_sales].astype('category')

# Extend for forecasting horizon
for i in range(horizon):
    df_sales[f'd_{last_day + i + 1}'] = np.nan

print("Sales:", df_sales.shape)

Sales: (30490, 1947)


## Melt and merge

In [8]:
# Melt Sales
# ===============================
id_vars = ['id'] + cat_cols_sales
df_sales = df_sales.melt(id_vars=id_vars, var_name='d', value_name='sold_qty')

In [9]:
# Merge Calendar
# ===============================
df_sales = df_sales.merge(df_calendar, on='d', how='left', copy=False)
del df_calendar
gc.collect()

# ===============================
# Merge Prices
# ===============================
df_sales = df_sales.merge(df_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left', copy=False)
del df_prices
gc.collect()

# ===============================
# Clean Columns
# ===============================
df_sales['day_ind'] = df_sales['d'].str.replace('d_', '').astype('int16')
df_sales.drop(['wm_yr_wk'], axis=1, inplace=True)  # keep weekday if needed

In [10]:
# Handle missing sell_price
# ===============================
df_sales = df_sales.sort_values(['store_id', 'item_id', 'day_ind'])

# Forward fill then backward fill per (store_id, item_id)
df_sales['sell_price'] = df_sales.groupby(['store_id', 'item_id'])['sell_price'].ffill()
df_sales['sell_price'] = df_sales.groupby(['store_id', 'item_id'])['sell_price'].bfill()

# If still missing (rare), fill with global median
df_sales['sell_price'] = df_sales['sell_price'].fillna(df_sales['sell_price'].median())

/tmp/ipykernel_47/1964641721.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sales['sell_price'] = df_sales.groupby(['store_id', 'item_id'])['sell_price'].ffill()
/tmp/ipykernel_47/1964641721.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sales['sell_price'] = df_sales.groupby(['store_id', 'item_id'])['sell_price'].bfill()


In [11]:
# Sort before saving
# ===============================
df_sales = df_sales.sort_values(['id', 'day_ind']).reset_index(drop=True)

In [12]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold_qty,date,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,day_ind
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,2011-01-29,Saturday,1,1,2011,NaN,NaN,NaN,NaN,False,False,False,2.0,1
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,2011-01-30,Sunday,2,1,2011,NaN,NaN,NaN,NaN,False,False,False,2.0,2
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,2011-01-31,Monday,3,1,2011,NaN,NaN,NaN,NaN,False,False,False,2.0,3
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,2011-02-01,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,True,True,False,2.0,4
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,2011-02-02,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,True,False,True,2.0,5


In [13]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sold_qty      float64       
 8   date          datetime64[ns]
 9   weekday       category      
 10  wday          int8          
 11  month         int8          
 12  year          int16         
 13  event_name_1  object        
 14  event_type_1  category      
 15  event_name_2  object        
 16  event_type_2  category      
 17  snap_CA       bool          
 18  snap_TX       bool          
 19  snap_WI       bool          
 20  sell_price    float32       
 21  day_ind       int16         
dtypes: bool(3), category(8), datetime64[ns](1), float32(1), float64(

## Save to parquet

In [14]:
# Save the consolidated dataset to Parquet

OUTPUT_PATH = "/kaggle/working/data_preprocessed.parquet"
df_sales.to_parquet(OUTPUT_PATH)

# Free memory
del df_sales
gc.collect()

# Print file size in MB
file_size_mb = Path(OUTPUT_PATH).stat().st_size / 1e6
print(f'Consolidated dataset size: {file_size_mb:,.2f} MB')


Consolidated dataset size: 274.50 MB
